# Python class linked with fortran module

In [1]:
%load_ext fortranmagic

In [2]:
%%fortran
module particles_f90

    implicit none
    
    real(8), dimension(:), allocatable :: positions
    real(8), dimension(:), allocatable :: velocities
            
contains
    subroutine init_particles( n )
    
        integer, intent(in) :: n
                
        integer :: i
        
        if (.not. allocated(positions)) then
            allocate(positions(n))
        end if
        positions = [(i, i = 1, n, 1)]
        if (.not. allocated(velocities)) then
            allocate(velocities(n))
        end if
        velocities = 1.0

    end subroutine init_particles
 
    subroutine push_particles( dt )
        
        real(8), intent(in) :: dt
    
        positions = positions + dt * velocities
        
    end subroutine push_particles
end module particles_f90

# The Python class

In [3]:
class Particles(object):
    
    def __init__(self, n):
        self.index       = 0
        self.numberof    = n
        particles_f90.init_particles( n)
        self.positions  = particles_f90.positions
        self.velocities = particles_f90.velocities
        
    @property 
    def position(self):      
        return self.positions[self.index]
    
    @property 
    def velocity(self):      
        return self.velocities[self.index]

# Access to Fortran data from Python

In [4]:
particles = Particles(10)
particles.velocities 

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [5]:
particles.positions

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [6]:
particles.index = 0
particles.position

1.0

In [7]:
particles.index = 1
particles.position

2.0

# Create an Iterator class

In [8]:
class ParticleArray(object):
    
    def __init__(self, particles):
        self.particles = particles
        self.numberof = particles.numberof
        
    def __getitem__(self, index): 
        self.particles.index = index 
        return self.particles
    
    def __len__(self): 
        return self.numberof
    
    def __iter__(self): 
        for i in range(self.numberof):
            self.particles.index = i
            yield self.particles

In [9]:
particle_array = ParticleArray(particles)
particle_array[0].position

1.0

In [10]:
for p in particle_array:
    print(p.position)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0


# Fortran derived type

In [11]:
%%fortran
module mesh

implicit none
type :: geometry
    real(8) :: xmin, xmax, dx            ! coordinates of origin and grid size
    integer :: nx                        ! number of grid points
    real(8), dimension(:), pointer :: x  ! coordinates of points
end type geometry

contains

subroutine create(geom, xmin, xmax, nx)

    !f2py integer(8), intent(out) :: geom
    type(geometry), pointer :: geom
    real(8), intent(in) :: xmin, xmax
    integer, intent(in) :: nx
            
    real(8) :: dx
            
    integer :: i
            
    allocate(geom)
    geom%xmin = xmin
    geom%xmax = xmax
    geom%dx = ( xmax - xmin ) / (nx-1) 
    geom%nx = nx
    allocate(geom%x(nx))
    do i=1,nx
        geom%x(i)=geom%xmin+(i-1)*geom%dx
    end do

end subroutine create

subroutine view(geom)
    !f2py integer(8), intent(in) :: geom
    type(geometry), pointer :: geom
    print*, 'nx = ', geom%nx
    print*, geom%xmin, geom%xmax
    print*, geom%x(:)
end subroutine view

subroutine get_size(geom, nx)

    !f2py integer(8), intent(in) :: geom
    type(geometry), pointer :: geom
    integer, intent(out) :: nx
    
    nx = geom%nx
    
end subroutine get_size


end module mesh

In [12]:
geom = mesh.create(0.0, 1.0, 10)

In [13]:
mesh.get_size(geom)

10

In [14]:
type(geom)

int

# f2py with C code

- Signature file is mandatory
- `intent(c)` must be used for all variables and can be set globally.
- Function name is declared with `intent(c)`

In [15]:
%rm -rf cfuncts*

In [16]:
%%file cfuncts.c

void push_particles(double* positions, double* velocities, double dt, int n){
    for (int i=0; i<n; i++){
       positions[i] = positions[i] + dt * velocities[i];
        
    }
} 

Writing cfuncts.c


In [17]:
%%file cfuncts.pyf

python module cfuncts 
    interface
        subroutine push_particles(positions, velocities, dt, n) 
            intent(c):: push_particles
            intent(c)
            integer, optional, depend(velocities) :: n = len(velocities)
            real(8), dimension(n),  intent(inplace)  :: positions 
            real(8), dimension(n),  intent(in) :: velocities
            real(8), intent(in) :: dt
        end subroutine push_particles
    end interface
end python module cfuncts

Writing cfuncts.pyf


In [18]:
%env FC=gfortran
%env CC=gcc-9
!f2py -c cfuncts.c cfuncts.pyf -m cfuncts

env: FC=gfortran
env: CC=gcc-9
running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "cfuncts" sources
creating /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpm79uzwsl/src.macosx-10.15-x86_64-3.7
f2py options: []
f2py: cfuncts.pyf
Reading fortran codes...
	Reading file 'cfuncts.pyf' (format:free)
Line #6 in cfuncts.pyf:"            intent(c) "
	All arguments will have attribute intent(c)
Post-processing...
	Block: cfuncts
			Block: push_particles
Post-processing (stage 2)...
Building modules...
	Building module "cfuncts"...
		Constructing wrapper function "push_particles"...
		  push_particles(positions,velocities,dt,[n])
	Wrote C/API module "cfuncts" to file "/var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpm79uzwsl/src.macosx-10.15-x86_64-3.7/cfunctsmodule.c"
  adding

In [19]:
import numpy as np
import cfuncts

In [20]:
print(cfuncts.push_particles.__doc__)

push_particles(positions,velocities,dt,[n])

Wrapper for ``push_particles``.

Parameters
----------
positions :  rank-1 array('d') with bounds (n)
velocities : input rank-1 array('d') with bounds (n)
dt : input float

Other Parameters
----------------
n : input int, optional
    Default: len(velocities)



In [21]:
n = 10
dt = 0.1
x = np.arange(n, dtype="d")
v = np.ones(n, dtype="d")
cfuncts.push_particles( x, v, dt)

In [22]:
x

array([0.1, 1.1, 2.1, 3.1, 4.1, 5.1, 6.1, 7.1, 8.1, 9.1])

# References
 
- f2py documentation https://docs.scipy.org/doc/numpy/f2py/
- Transparents E. Sonnendrucker http://calcul.math.cnrs.fr/Documents/Journees/dec2006/python-fortran.pdf
- Documentation Sagemath http://doc.sagemath.org/html/en/thematic_tutorials/numerical_sage/f2py.html
- Hans Petter Langtangen : Python Scripting for Computational Science.